In [1]:
#Sytem
import sys,os
import numpy as np
import pandas as pd
import warnings
import time
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



#model
from sklearn import ensemble
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from skopt import BayesSearchCV
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier


print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)


----------------------------------------
Python version: 3.9.10 (main, Feb 22 2022, 13:54:07) 
[GCC 11.2.0]
pandas version: 1.3.5
matplotlib version: 3.5.1
NumPy version: 1.21.5
scikit-learn version: 1.1.1
----------------------------------------


In [2]:
Target = "Perished"
PATH_LOG = "./log.csv"
PATH_df = "./df"
ID = 10
SEED = 42

In [3]:
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
log = pd.read_csv(PATH_LOG)
data = pd.concat([df_train,df_test],sort=False)
print('-'*25)
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print('-'*25)
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print('-'*25)

-------------------------
Shape of Train Data
[row :891,column :12]
-------------------------
Shape of Test Data
[row :418,column :11]
-------------------------


In [4]:
label = LabelEncoder()
ohe = OneHotEncoder()

In [5]:
data['Title'] = data['Name'].str.split(
        ", ", expand=True)[1].str.split(".", expand=True)[0]

title_names = (data.Title.value_counts() < 10)
data["Title"] = data["Title"].apply(lambda x: "other" if title_names.loc[x] == True else x)

In [6]:
for title in data.Title.unique():
    mean = data.loc[(data.Title.values == title),"Age"].mean()
    data.loc[(data.Title.values == title) & (data.Age.isnull()),"Age"] = mean

In [7]:
faremean = data.Fare.mean()
data["Fare"].fillna(faremean,inplace=True)
data['FareBin'] = pd.qcut(data['Fare'], 10)

In [8]:
N_Age_Code = 3
data.replace({'Sex':{'male':0,'female':1}},inplace=True)
data['Age_Code'] = label.fit_transform(pd.qcut(data['Age'], N_Age_Code))
data['Embarked_Code'] = label.fit_transform(data['Embarked'])

In [9]:
data["Embarked"].fillna('S',inplace=True)
embarked_ohe = pd.get_dummies(data.Embarked)
data = pd.concat([data, embarked_ohe], axis=1)

In [10]:
drop_columns = ["Name","Ticket","Cabin","Embarked","Title",'FareBin']
dataset = data.drop(columns=drop_columns)
train = dataset[:len(df_train)]
test = dataset[len(df_train):]

In [11]:
train.head(2)

,PassengerId,Perished,Pclass,Sex,Age,SibSp,Parch,Fare,Age_Code,Embarked_Code,C,Q,S
0,1,1.0,3,0,22.0,1,0,7.2500,0,2,0,0,1
1,2,0.0,1,1,38.0,1,0,71.2833,2,0,1,0,0


In [12]:
X = train.iloc[:, 2:].values
y = train.iloc[:, 1].values
X_test = test.iloc[:, 2:].values

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
vote_est = [
    #Ensemble Methods: http://scikit-learn.org/stable/modules/ensemble.html
    ('ada', ensemble.AdaBoostClassifier()),
    ('bc', ensemble.BaggingClassifier()),
    ('etc',ensemble.ExtraTreesClassifier()),
    ('gbc', ensemble.GradientBoostingClassifier()),

    #Gaussian Processes: http://scikit-learn.org/stable/modules/gaussian_process.html#gaussian-process-classification-gpc
    ('gpc', gaussian_process.GaussianProcessClassifier()),
    
    #GLM: http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
    ('lr', linear_model.LogisticRegressionCV()),
    
    #Navies Bayes: http://scikit-learn.org/stable/modules/naive_bayes.html
    ('bnb', naive_bayes.BernoulliNB()),
    ('gnb', naive_bayes.GaussianNB()),
    
    #Nearest Neighbor: http://scikit-learn.org/stable/modules/neighbors.html
    ('knn', neighbors.KNeighborsClassifier()),
    
    #SVM: http://scikit-learn.org/stable/modules/svm.html
    ('svc', svm.SVC(probability=True)),
    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
   ('xgb', XGBClassifier())

]

In [18]:
Modelname = "Emsenble"
CV = 3
grid_hard = ensemble.VotingClassifier(estimators = vote_est , voting = 'hard')
grid_hard_cv = model_selection.cross_validate(grid_hard, X_train,y_train, cv  = CV,return_train_score=True)
grid_hard.fit(X_train, y_train)

VotingClassifier(estimators=[('ada', AdaBoostClassifier()),
                             ('bc', BaggingClassifier()),
                             ('etc', ExtraTreesClassifier()),
                             ('gbc', GradientBoostingClassifier()),
                             ('gpc', GaussianProcessClassifier()),
                             ('lr', LogisticRegressionCV()),
                             ('bnb', BernoulliNB()), ('gnb', GaussianNB()),
                             ('knn', KNeighborsClassifier()),
                             ('svc', SVC(probability=True)),
                             ('xgb',
                              XGBClassifier(base_s...
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None,
                                            reg_alpha=None, reg_lambda=None, ...))])

In [19]:
print("Hard Voting w/Tuned Hyperparameters Training w/bin score mean: {:.2f}". format(grid_hard_cv['train_score'].mean()*100)) 
print("Hard Voting w/Tuned Hyperparameters Test w/bin score mean: {:.2f}". format(grid_hard_cv['test_score'].mean()*100))
print("Hard Voting w/Tuned Hyperparameters Test w/bin score 3*std: +/- {:.2f}". format(grid_hard_cv['test_score'].std()*100*3))
print('-'*10)

Hard Voting w/Tuned Hyperparameters Training w/bin score mean: 92.35
Hard Voting w/Tuned Hyperparameters Test w/bin score mean: 80.20
Hard Voting w/Tuned Hyperparameters Test w/bin score 3*std: +/- 4.61
----------


In [20]:
pred = grid_hard.predict(X_test)

In [21]:
PATH_sub =  ("./sub/"+str(ID)+"_submission_"+Modelname+".csv")
print(PATH_sub)
submission_ex = pd.read_csv("gender_submission.csv")
submission_df = pd.DataFrame(columns=['PassengerId', Target])
submission_df['PassengerId'] = df_test['PassengerId']
submission_df[Target] = pred
submission_df.reset_index(drop=True, inplace=True)
submission_df.to_csv(PATH_sub, header=True, index=False)
print("Submission Format:{} ".format(submission_ex.shape))
print("My Sumbisison Format:{}".format(submission_df.shape))
submission_df.head(5)

./sub/10_submission_Emsenble.csv
Submission Format:(418, 2) 
My Sumbisison Format:(418, 2)


,PassengerId,Perished
0,892,1.0
1,893,1.0
2,894,1.0
3,895,1.0
4,896,1.0
